In [1]:
from func import *

In [2]:
raw_df = load_csvs_from_ftp_to_df(provided_local_dir="/")
processed_df = pre_process(raw_df)
df_clean = processed_df.copy(deep=True)

In [3]:
quality_summary_table = data_quality_summary(processed_df)
quality_summary_table.sort_values(by=["missing_%"], ascending=False)

,column,dtype,n_unique,missing_%,num_missing
36,BelowGradeFinishedArea,float64,42,99.29,67337
23,BuilderName,object,812,95.41,64707
43,LotSizeDimensions,object,2780,93.62,63493
31,BuildingAreaTotal,float64,2444,93.47,63389
20,ElementarySchool,object,1124,86.79,58859
38,MiddleOrJuniorSchool,object,585,86.57,58708
41,HighSchool,object,476,82.63,56038
17,AssociationFeeFrequency,object,4,75.00,50865
16,CoListOfficeName,object,2346,73.42,49793
25,SubdivisionName,object,5638,64.52,43755


In [4]:
# Drop "BuilderName" and "LotSizeDimensions" due to high rate of missing
df_clean.drop(["BuilderName", "LotSizeDimensions"], inplace=True, axis=1)

In [5]:
# range for latitude: between 31 and 42, longitude between -125 and -115
df_clean = df_clean[(df_clean["Latitude"].between(31, 42) ) & (df_clean["Longitude"].between(-125, -115))]

In [6]:
# For "City", "SubdivisionName", Nearest-neighbor assignment through longitude and latitude
df_clean = fill_na_by_cluster(df=df_clean,
                              target_col_list=["City", "SubdivisionName"],
                              reference_col_list=["Longitude", "Latitude"],
                              method="knn")

In [7]:
# For "Levels", "Flooring", using binary flag, fill the na with false
df_clean = create_flag(df_clean, ["Levels", "Flooring"])

C:\My_Programs\Python_Project\IDX_Intern\func.py:242: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy[val + "YN"] = df_copy[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern\func.py:242: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy[val + "YN"] = df_copy[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern\func.py:242: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the futur

In [8]:
# For "CoListOfficeName", "MLSAreaMajor", "BuyerOfficeAOR", "BuyerOfficeName", "EmailDomain", fill with mode by city
df_clean = fill_na_with_mode(df=df_clean,
                             target_col_list=["CoListOfficeName", "MLSAreaMajor", "BuyerOfficeAOR", "BuyerOfficeName", "EmailDomain"],
                             reference_col="City")

In [9]:
# For school-related variables, Nearest-neighbor assignment through longitude and latitude
df_clean = fill_na_by_cluster(df=df_clean,
                              target_col_list=["ElementarySchool", "MiddleOrJuniorSchool", "HighSchool", "HighSchoolDistrict"],
                              reference_col_list=["Longitude", "Latitude"],
                              method="knn")

In [10]:
# For "YN" variables, fill the na with False
df_clean["AttachedGarageYN"] = df_clean["AttachedGarageYN"].fillna(False)
df_clean["ViewYN"] = df_clean["ViewYN"].fillna(False)
df_clean["NewConstructionYN"] = df_clean["NewConstructionYN"].fillna(False)
df_clean["PoolPrivateYN"] = df_clean["PoolPrivateYN"].fillna(False)
df_clean["FireplaceYN"] = df_clean["FireplaceYN"].fillna(False)

C:\Users\lee39\AppData\Local\Temp\ipykernel_9668\2920445219.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean["AttachedGarageYN"] = df_clean["AttachedGarageYN"].fillna(False)
C:\Users\lee39\AppData\Local\Temp\ipykernel_9668\2920445219.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean["ViewYN"] = df_clean["ViewYN"].fillna(False)
C:\Users\lee39\AppData\Local\Temp\ipykernel_9668\2920445219.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer

In [11]:
# Drop "UnparsedAddress"
df_clean.drop(["UnparsedAddress"], axis=1, inplace=True)

In [12]:
# For "BuyerAgentAOR" and "ListAgentAOR", fill na with other
df_clean["BuyerAgentAOR"] = df_clean["BuyerAgentAOR"].fillna("Other")
df_clean["ListAgentAOR"] = df_clean["ListAgentAOR"].fillna("Other")

In [13]:
# Drop "PurchaseContractDate", not related with "ClosePrice", since we already have "CloseDate"
df_clean.drop(["PurchaseContractDate"], axis=1, inplace=True)

In [14]:
# For other variables, clustering according to all variables, then fill the na
quality_summary_table = data_quality_summary(df_clean)
col_need_to_fill_na = quality_summary_table[quality_summary_table["num_missing"] > 0]["column"]
reference_col_list = quality_summary_table[quality_summary_table["num_missing"] == 0]["column"]

In [15]:
# reference_col_list = ["ClosePrice", "City", "CountyOrParish"]

In [16]:
df_clean = fill_na_by_cluster(df=df_clean,
                              target_col_list=col_need_to_fill_na,
                              reference_col_list=reference_col_list,
                              method="k-prototypes",
                              num_clusters=10,
                              random_state=42)

In [19]:
quality_summary_table = data_quality_summary(df_clean)
quality_summary_table.sort_values(by=["missing_%"], ascending=False)

,column,dtype,n_unique,missing_%,num_missing
0,BuyerAgentAOR,object,53,0.0,0
1,ListAgentAOR,object,53,0.0,0
2,ViewYN,bool,2,0.0,0
3,PoolPrivateYN,bool,2,0.0,0
4,OriginalListPrice,float64,6421,0.0,0
...,...,...,...,...,...
56,BambooYN,bool,2,0.0,0
57,ConcreteYN,bool,2,0.0,0
58,StoneYN,bool,2,0.0,0
59,SeeRemarksYN,bool,2,0.0,0


In [21]:
save_name = "cleaned"
i = 1
while os.path.exists(save_name + ".csv"):
    save_name = save_name + str(i)
    i += 1
df_clean.to_csv(save_name + ".csv", index=False)